In [2]:
import pandas as pd
import h5py  
import numpy as np

In [4]:
#Load data from h5py
hdf5_path = '/Users/GuoZhiqi/Downloads/sub1_s99_sg.hdf5' 
data = h5py.File(hdf5_path,'r') 

In [5]:
def print_keys(obj):
    print([key for key in obj.keys()]) 

In [6]:
print_keys(data) 

['Header', 'PartType0', 'PartType4']


In [7]:
PartType0 = data['PartType0']  #Gas
print_keys(PartType0) 

['Coordinates', 'FOFID', 'GalID', 'Masses', 'ParticleIDs', 'StarFormationRate', 'Velocities']


In [8]:
PartType4 = data['PartType4'] #Star
print_keys(PartType4)  

['Coordinates', 'FOFID', 'GFM_StellarFormationTime', 'GFM_StellarPhotometrics', 'GalID', 'Masses', 'ParticleIDs', 'Velocities']


In [91]:
def cal_angular_mom(dataset,Type,alternative_weighted = False,cutoff = False):
    '''
    Formula: |sum j| = |sum mrv|/sum of mass
    
    The Type should be either:
    1).Gas or 2).Star
    
    1. Calculate unweighted augular momentum for Gaseous Components(PartType0 - GAS) 
    2. Calculate weighted augular momentum for Gaseous Components(PartType0 - GAS)
        (instead of mass use StarFormationRate) 
    3. Calculate unweighted augular momentum for stellar Components(PartType4 - STARS & WIND PARTICLES)
    To do : alternative
    
    '''
    if (Type == 'Gas'):
        dataset = dataset['PartType0']
        PartType0_aug_related_columns = np.concatenate([dataset['Coordinates'][:], 
                                                        dataset['Masses'][:], 
                                                        dataset['StarFormationRate'][:],
                                                        dataset['Velocities'][:]],
                                                        axis=1)
        if(cutoff):
            cut_off_radius = [np.linalg.norm(particle_radius)<=20 for particle_radius in PartType0_aug_related_columns[:,:3]]
            PartType0_aug_related_columns = PartType0_aug_related_columns[cut_off_radius]
            
        Coordinates = PartType0_aug_related_columns[:,0:3]
        Masses = PartType0_aug_related_columns[:,3]
        StarFormationRate = PartType0_aug_related_columns[:,4]
        Velocities = PartType0_aug_related_columns[:,5:8]
        
        if(alternative_weighted):
            Weight = StarFormationRate
        else:
            Weight = Masses
            
        part_angular_momentum = np.multiply(Weight.reshape(len(Weight),1),np.cross(Coordinates,Velocities))
        sum_angular_momentum = [np.sum([x[0] for x in part_angular_momentum]),np.sum([x[1] for x in part_angular_momentum]),np.sum([x[2] for x in part_angular_momentum])]
        if(np.sum(Weight)!=0):
            result = np.linalg.norm(sum_angular_momentum)/np.sum(Weight) #L2 norm
        else:
            result = 0
        return result
        
    if (Type == 'Star') & (not alternative_weighted):
        '''
        Only consider particles which GFM_StellarFormationTime >=0 
        '''
        #Filter out real stars (GFM_StellarFormationTime >=0 )
        dataset =dataset['PartType4']
        PartType4_aug_related_columns = np.concatenate([dataset['Coordinates'][:], 
                                                        dataset['Masses'][:], 
                                                        dataset['GFM_StellarFormationTime'][:],
                                                        dataset['Velocities'][:],
                                                        dataset['GFM_StellarPhotometrics'][:]],
                                                        axis=1)
        PartType4_aug_related_columns = PartType4_aug_related_columns[PartType4_aug_related_columns[:,4]>=0 ]
        if(cutoff):
            cut_off_radius = [np.linalg.norm(particle_radius)<=20 for particle_radius in PartType4_aug_related_columns[:,:3]]
            PartType4_aug_related_columns = PartType4_aug_related_columns[cut_off_radius]
        
        Coordinates = PartType4_aug_related_columns[:,0:3]
        Masses = PartType4_aug_related_columns[:,3]
        GFM_StellarFormationTime = PartType4_aug_related_columns[:,4]
        Velocities = PartType4_aug_related_columns[:,5:8]
        GFM_StellarPhotometrics = PartType4_aug_related_columns[:,8:]
        
        part_angular_momentum = np.multiply(Masses.reshape(len(Masses),1),np.cross(Coordinates,Velocities))
        sum_angular_momentum = [np.sum([x[0] for x in part_angular_momentum]),np.sum([x[1] for x in part_angular_momentum]),np.sum([x[2] for x in part_angular_momentum])]
        return np.linalg.norm(sum_angular_momentum)/np.sum(Masses) #L2 norm 
        

In [63]:
print('Starformation weighted augular momentum for Gas:',cal_angular_mom(data,'Gas',True,False))  

Starformation weighted augular momentum for Gas: 114853.0


In [58]:
print('Masses weighted augular momentum for Gas:',cal_angular_mom(data,'Gas',False,False))  

Masses weighted augular momentum for Gas: 30939.5


In [59]:
print('Masses weighted augular momentum for Gas with Cutoff:',cal_angular_mom(data,'Gas',False,True))  

Masses weighted augular momentum for Gas with Cutoff: 1319.87


In [60]:
print('mass weighted augular momentum for Star :',cal_angular_mom(data,'Star',False,False))

mass weighted augular momentum for Star : 11189.4


In [89]:
print('mass weighted augular momentum for Star with Cutoff:',cal_angular_mom(data,'Star',False,True))

mass weighted augular momentum for Star with Cutoff: 504.618


In [ ]:
print('Starformation weighted augular momentum for Gas with Cutoff:',cal_angular_mom(data,'Gas',True,True))  